In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from pyod.models.knn import KNN

In [2]:
data = pd.read_feather('database/base.feather')

In [3]:
def rfm_variables(df):
    f_sales = len(df)
    f_profit = len(df[df['Profit'] > 0])
    m_sales = round(df['Sales'].sum(),2)
    m_profit = round(df['Profit'].sum(),2)
    m_qty = df['Quantity'].sum()
    df_sort = df[['Order Date']].sort_values(by='Order Date').drop_duplicates()
    df_sort['diff'] = df_sort['Order Date'] - df_sort['Order Date'].shift(1)
    df_sort['diff_int'] = df_sort['diff'].dt.days
    r_days = round(df_sort['diff_int'].mean(),2)
    return f_sales, f_profit, m_sales, m_profit, m_qty, r_days

In [4]:
def fit_data(data, variable):
    rfm = pd.DataFrame()
    variaveis = data[variable].unique()
    for variavel in variaveis:
        dados = data[data[variable] == variavel]
        f_vendas, f_lucro, m_vendas, m_lucro, m_qtde, r_dias = rfm_variables(dados)
        new_row = {
            'referencia': variavel,
            'm_vendas': m_vendas,
            'm_lucro': m_lucro,
            'm_qtde': m_qtde,
            'r_dias': r_dias,
            'f_vendas': f_vendas,
            'f_lucro': f_lucro
        }
        rfm = rfm.append(
            new_row,
            ignore_index = True
        )
    return rfm

In [5]:
def outliers_detection(orig, vars):
    sc_x = StandardScaler()
    datax = sc_x.fit_transform(vars)
    clf = KNN().fit(vars)
    outliers = clf.predict(vars)
    dataod = orig.copy()
    dataod['outlier'] = outliers
    return dataod

In [6]:
variaveis = ['m_vendas', 'm_lucro', 'm_qtde', 'r_dias', 'f_vendas', 'f_lucro']

In [7]:
## Aula 16 - Outliers
outliers_pais = fit_data(data, 'State')
outliers_pais = outliers_pais.fillna(0)
outlier = outliers_pais[variaveis].fillna(0).copy()
outliers = outliers_detection(outliers_pais, outlier)
outliers

/tmp/ipykernel_2500/2583580270.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
/tmp/ipykernel_2500/2583580270.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
/tmp/ipykernel_2500/2583580270.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
/tmp/ipykernel_2500/2583580270.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
/tmp/ipykernel_2500/2583580270.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
/tmp/ipykernel_2500/2583580270.py:16: FutureW

,referencia,m_vendas,m_lucro,m_qtde,r_dias,f_vendas,f_lucro,outlier
0,Kentucky,36591.75,11199.70,523,24.63,139,138,0
1,California,457687.63,76381.39,7667,2.21,2001,1884,1
2,Florida,89473.71,-3399.30,1379,7.75,383,252,0
3,North Carolina,55603.16,-7490.91,983,10.66,249,180,0
4,Washington,138641.27,33402.65,1883,6.54,506,483,1
5,Texas,170188.05,-25729.36,3724,3.73,985,493,1
6,Wisconsin,32114.61,8401.80,463,27.23,110,110,0
7,Utah,11220.06,2546.53,219,53.16,53,53,0
8,Nebraska,7464.93,2037.09,136,57.00,38,38,0
9,Pennsylvania,116511.91,-15559.96,2153,5.70,587,303,1


In [ ]:
outliers.to_feather('database/outliers_estado.feather')